написать класс для запросов

Do not contact the server more often than once every 10 seconds.\
Do not poll for any single RID more often than once a minute.\
Use the URL parameter email and tool, so that the NCBI can contact you if there is a problem.\
Run scripts weekends or between 9 pm and 5 am Eastern time on weekdays if more than 50 searches will be submitted.


If your queries are short (less than a few hundred bases) we suggest you merge them into one search of up to 1,000 bases


# пример - https://github.com/biopython/biopython/blob/master/Bio/Blast/NCBIWWW.py

# Добавить
 ## поиск taxid по названию (выбирать по умолчанию один самый совпадающий?)
 https://ena-docs.readthedocs.io/en/latest/faq/taxonomy.html тут можно посмотреть удачный способ это сделать
 
 ## вещи которые мог менять древний апи бласта 
 https://ncbi.github.io/blast-cloud/dev/api.html

In [172]:
import urllib
import requests
import time

from bs4 import BeautifulSoup

Пример использования:

    - последовательность: 
        "MVMFGNKVSYPYMDLSPKSHWIVYEGNLKFYDKQGRDTASIKDAASLSFDFFTWGRNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT",
    - база данных: Whole_Genome_Shotgun_contigs,
    - организм: Prevotella sp. oral taxon 472 str. F0295 (taxid:619693),
    - tblastn.

In [163]:
# поиск организма в базе данных WGS https://www.ncbi.nlm.nih.gov/Traces/wgs/

In [66]:
WGS_PROJECTS_URL = 'https://www.ncbi.nlm.nih.gov/Traces/wgs/index.cgi?'

In [ ]:
WGS_payload = {
}

text:*Prevotellaceae* AND project_s:wgs AND (project_s:WGS) AND (project_s:WGS)

In [ ]:
# генерация уникального имени - https://docs.python.org/3/library/uuid.html
import uuid

# make a random UUID
uuid.uuid4()

# make a UUID from a string of hex digits (braces and hyphens ignored)
x = uuid.UUID('{00010203-0405-0607-0809-0a0b0c0d0e0f}')

# рабочий простой вариант

# запрос в ncbi

In [152]:
fasta = \
""">line1
MVMFGNKVSYPYMDLSPKSHWIVYEGNLKFYDKQGRDTASIKDAASLSFDFFTWGRNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT
"""

In [153]:
BLAST_URL = "https://blast.ncbi.nlm.nih.gov/Blast.cgi"

In [189]:
payload = {
    'QUERY': fasta,
    'db': 'protein',
    #'QUERYFILE': '(binary)',  # use for binary files
    'GENETIC_CODE': 1,
    'JOB_TITLE': 'fasta',
    'ADV_VIEW': 'true',
    'stype': 'nucleotide',
    #'SUBJECTFILE': '(binary)', # use for binary files
    'DATABASE': 'Whole_Genome_Shotgun_contigs',
    'DB_GROUP': 'wgsOrg',
    #'EQ_MENU': 'Prevotella sp. oral taxon 472 str. F0295 (taxid:619693)',  # работает если передавать полное название из выпадающего списка
    'EQ_MENU': '619693',    # работает и просто при передаче самого айдишника
    #'EQ_MENU': 'Prevotella sp. oral taxon 472 str. F0295',    # а так не работает
    'NUM_ORG': 1,
    'MAX_NUM_SEQ': 100,
    'EXPECT': 0.05,
    'WORD_SIZE': 6,
    'HSP_RANGE_MAX': 0,
    'MATRIX_NAME': 'BLOSUM62',
    'MATCH_SCORES': [1, -2],
    'GAPCOSTS': '11 1',
    'COMPOSITION_BASED_STATISTICS': 2,
    'FILTER': 'L',
    'REPEATS': 566037,
    'TEMPLATE_LENGTH': 0,
    'TEMPLATE_TYPE': 0,
    'PSSM': '(binary)',
    'SHOW_OVERVIEW': 'true',
    'SHOW_LINKOUT': 'true',
    'GET_SEQUENCE': 'true',
    'FORMAT_OBJECT': 'Alignment',
    'FORMAT_TYPE': 'HTML',
    'ALIGNMENT_VIEW': 'Pairwise',
    'MASK_CHAR': 2,
    'MASK_COLOR': 1,
    'DESCRIPTIONS': 100,
    'ALIGNMENTS': 100,
    'LINE_LENGTH': 60,
    'NEW_VIEW': 'true',
    'NCBI_GI': 'false',
    'SHOW_CDS_FEATURE': 'false',
    'NUM_OVERVIEW': 100,
    'FORMAT_NUM_ORG': 1,
    'CONFIG_DESCR': '2,3,4,5,8,9,10,11,12,13,14',
    'CLIENT': 'web',
    'SERVICE': 'plain',
    'CMD': 'request',
    'PAGE': 'Translations',
    'PROGRAM': 'tblastn',
    'UNGAPPED_ALIGNMENT': 'no',
    'BLAST_PROGRAMS': 'tblastn',
    'DB_DISPLAY_NAME': 'wgs',
    'ORG_DBS': 'orgDbsOnly_wgs',
    'SHOW_ORGANISMS': 'on',
    'SELECTED_PROG_TYPE': 'tblastn',
    'SAVED_SEARCH': 'true',
    'NUM_DIFFS': 1,
    'NUM_OPTS_DIFFS': 0,
    #'UNIQ_DEFAULTS_NAME': 'A_SearchDefaults_1nRG8I_3Rxm_degTm11k3CN_GTXQl_1Twy123r',
    'PAGE_TYPE': 'BlastSearch',
    #'USER_DEFAULT_PROG_TYPE': 'tblastn'
}

In [224]:
post_time = time.time()
resp = requests.post(BLAST_URL, data=payload)

soup = BeautifulSoup(resp.content)
resp.status_code

200

In [225]:
# Extract RID
RID = soup.find('input', {'name': 'RID'}).get('value')
RID

'2ETE2J2S01R'

## запрашиваем результат по риду, пока не получим его

You may check the status of the search by invoking blast.cgi with the parameters CMD=Get, FORMAT_OBJECT=SearchInfo, and RID=VALUE, where VALUE is the RID returned by the server. The server will respond with a page containing a QBlastInfo block with the status set as one of WAITING, UNKNOWN, or READY.


Once the search is finished, you may retrieve results by invoking blast.cgi with the parameters CMD=GET and RID=VALUE, where VALUE is the RID the system returned. An example would be “CMD=Get&RID=6NTRF1YLO8”. You may also specify the report type (e.g, add FORMAT_TYPE=Text).



In [226]:
payload = {
    'CMD': 'Get',
    'FORMAT_OBJECT': 'SearchInfo',
    'RID': RID
}

In [227]:
resp = requests.post(BLAST_URL, data=payload)

soup = BeautifulSoup(resp.content)
resp.status_code

200

In [231]:
soup.select_one('td:contains("Status") + td').text.upper()

'SEARCHING'

In [228]:
delay = 11
current_time = time.time()
wait = post_time + delay - current_time

while True:
    
    if wait > 0:
        time.sleep(wait)
        current_time = time.time
    
    else:
        
        if soup.select_one('td:contains("Status") + td').text.upper() == 'READY':
            break
            
        resp = requests.post(BLAST_URL, data=payload)
        soup = BeautifulSoup(resp.content)
        resp.status_code
            
    

KeyboardInterrupt: 

## приводим результаты в читаемый вид

In [141]:
soup.contents

['html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"',
 <html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
 <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
 <meta content="ncbitoggler ncbiautocomplete" name="jig"/>
 <meta content="animation:'none'" name="ncbitoggler"/>
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 <meta content="blast2seq,Align two sequences using BLAST (bl2seq)" name="keywords"/>
 <meta content="origin-when-cross-origin" name="referrer"/>
 <meta content="blast" name="ncbi_app"/>
 <meta content="blastsearch" name="ncbi_pdid"/>
 <meta content="" name="ncbi_db"/>
 <meta content="tblastn" name="ncbi_program"/>
 <meta content="" name="ncbi_algorithm"/>
 <meta content="false" name="ncbi_stat"/>
 <meta content="55BA67A7227417D1_0000SID" name="ncbi_sessionid"/>
 <meta content="55BA67A7227417D10000000000000001" name="ncbi_phid"/>
 <meta content="" name=

In [176]:
for header in resp.headers:
    print(header)

Date
Server
Strict-Transport-Security
Referrer-Policy
Content-Security-Policy
Cache-Control
NCBI-PHID
NCBI-SID
Content-Type
Set-Cookie
X-UA-Compatible
X-XSS-Protection
Keep-Alive
Connection
Transfer-Encoding


In [11]:
print(resp.headers['NCBI-PHID'])
print(resp.headers['NCBI-SID'])


55BA0214227219510000000000000001.m_1
55BA021422721951_0000SID


In [109]:
payload_2 = {
    'ALIGNMENTS': 100,
    'ALIGNMENT_VIEW': 'Pairwise',
    'BLAST_PROGRAMS': 'tblastn',
    'DB_GROUP': 'wgsOrg',
    'DESCRIPTIONS': 100,
    'EQ_MENU': 'taxid:619693',
    'EXPECT': 0.05,
    'FORMAT_OBJECT': 'Alignment',
    'FORMAT_TYPE': 'HTML',
    'GET_SEQUENCE': 'true',
    'JOB_TITLE': 'fasta',
    'LINE_LENGTH': 60,
    'MATRIX_NAME': 'BLOSUM62',
    'MAX_NUM_SEQ': 100,
    'ORG_DBS': 'orgDbsOnly_wgs',
    'PAGE': 'Translations',
    'SELECTED_PROG_TYPE': 'tblastn', 
    'REPEATS': 566037,
    'WORD_SIZE': 6,
    'db': 'protein',
    'stype': 'nucleotide',
    'UNIQ_DEFAULTS_NAME': 'A_SearchDefaults_1nRG8I_3Rxm_degTm11k3CN_GTXQl_1sfbvb3',
    
    'PAGE_TYPE': 'BlastSearch',
    'SAVED_SEARCH': 'true',
    'SELECTED_PROG_TYPE': 'tblastn',
    'CMD': 'Get'
}

In [110]:
resp_2 = requests.post(form_url, data=payload_2)
soup_2 = BeautifulSoup(resp_2.content)
resp_2.status_code

200

In [108]:
print(resp.headers['NCBI-PHID'])
print(resp.headers['NCBI-SID'])


50C965D6226635210000000000000001.m_1
50C965D622663521_0000SID


In [112]:
print(resp_2.headers['NCBI-PHID'])
print(resp_2.headers['NCBI-SID'])

50C920A4226637010000000000000001.m_1
50C920A422663701_0000SID


In [223]:
def save_content(resp, filename):
    with open(filename, "wb") as file:
        file.write(resp.content)
        
save_content(resp, "page.html")

In [107]:
def save_content(resp, filename):
    with open(filename, "wb") as file:
        file.write(resp.content)
        
save_content(resp, "page.html")

In [34]:
page = urllib.request.urlopen("https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=tblastn&PAGE_TYPE=BlastSearch&BLAST_SPEC=&LINK_LOC=blasttab&LAST_PAGE=tblastn&QUERY=cgbdgfh")
soup = BeautifulSoup(page)

In [35]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="ncbitoggler ncbiautocomplete" name="jig"/>
<meta content="animation:'none'" name="ncbitoggler"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="blast2seq,Align two sequences using BLAST (bl2seq)" name="keywords"/>
<meta content="origin-when-cross-origin" name="referrer"/>
<meta content="blast" name="ncbi_app"/>
<meta content="blastsearch" name="ncbi_pdid"/>
<meta content="" name="ncbi_db"/>
<meta content="tblastn" name="ncbi_program"/>
<meta content="" name="ncbi_algorithm"/>
<meta content="false" name="ncbi_stat"/>
<meta content="50C9463E2262D7D1_0000SID" name="ncbi_sessionid"/>
<meta content="50C9463E2262D7D10000000000000001" name="ncbi_phid"/>
<meta content="" name="ncbi_pci

последовательность для поиска нужно вводить вместо "cgbdgfh":

<textarea class="reset" cols="80" id="seq" name="QUERY" rows="5" suggesthint="">cgbdgfh</textarea>


база данных wgs

<option abbr="wgs" dbgroup="Rgen" dynjs="initLimitType('WGS')" hide="orgLbl" orgdbs="orgDbsOnly_wgs" orgdict="bdb_wgs_all_sg" show="wgsLimitBy" showorg="on" value="Whole_Genome_Shotgun_contigs">Whole-genome shotgun contigs (wgs)</option>


выбор организма 


<input name="EQ_MENU" size="55" type="text" id="qorganism" value="" dicttype="taxid" data-jigconfig="dictionary:'taxids_sg',isCrossDomain:false" autocomplete="off" data-jig="ncbiautocomplete" class="reset" title="Use up and down arrows to choose an item from the autocomplete." aria-haspopup="true" aria-autocomplete="list" role="textbox" suggesthint="Enter organism name or id--completions will be uggested" topoption="" valueid="human 9606">




In [36]:
for link in soup.find_all('a'):
    print(link.get('href'))

https://www.ncbi.nlm.nih.gov/
https://www.ncbi.nlm.nih.gov/myncbi
#mainCont
#navcontent
https://www.ncbi.nlm.nih.gov/guide/browsers/#accesskeys
https://www.ncbi.nlm.nih.gov/
https://www.ncbi.nlm.nih.gov/account/?back_url=https%3A%2F%2Fblast%2Encbi%2Enlm%2Enih%2Egov%2FBlast%2Ecgi%3FBLAST%5FPROGRAMS%3Dtblastn%26BLAST%5FSPEC%3D%26LAST%5FPAGE%3Dtblastn%26LINK%5FLOC%3Dblasttab%26MYNCBI%5FUSER%3D11556657473%26MYNCBI%5FUSER%3D11556657473%26PAGE%5FTYPE%3DBlastSearch%26PROGRAM%3Dtblastn%26QUERY%3Dcgbdgfh%26USER%5FTYPE%3D2%26USER%5FTYPE%3D2
https://www.ncbi.nlm.nih.gov/myncbi/
https://www.ncbi.nlm.nih.gov/myncbi/collections/bibliography/
https://www.ncbi.nlm.nih.gov/account/settings/
https://www.ncbi.nlm.nih.gov/account/signout/?back_url=https%3A%2F%2Fblast%2Encbi%2Enlm%2Enih%2Egov%2FBlast%2Ecgi%3FBLAST%5FPROGRAMS%3Dtblastn%26BLAST%5FSPEC%3D%26LAST%5FPAGE%3Dtblastn%26LINK%5FLOC%3Dblasttab%26MYNCBI%5FUSER%3D11556657473%26MYNCBI%5FUSER%3D11556657473%26PAGE%5FTYPE%3DBlastSearch%26PROGRAM%3Dtblastn%

In [ ]:
class TBlastN():
    '''
    Class for search in tblastn database.
    '''
    
    def __init__(self):
        
        
        
    def tn_search(self, sequence, database='WGS', organism):
        '''
        функцию для запроса, которая будет принимать 3 обязательных аргумента: 
        белковая последовательность, которую мы бластим, 
        базу данных (в этом задании нас интересует только WGS, но по желанию можете добавить что-нибудь ещё), 
        таксон, у которого мы ищем последовательность, чаще всего - конкретный вид.
        
        Функция дожна возвращать список объектов типа Alignment, у него должны быть следующие атрибуты
        self.score
        self.subj_range    # tuple
        self.subj_name
        self.subj_id
        self.subj_len
        self.e_value
        self.identity
        self.query_seq
        self.subj_seq
        
        '''
        
        return self.alingment
    
    
    
    def get_aligment(self):
        
        self.score
        self.subj_range    # tuple
        self.subj_name
        self.subj_id
        self.subj_len
        self.e_value
        self.identity
        self.query_seq
        self.subj_seq
        
    